In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [2]:
import sklearnex
sklearnex.patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
data=pd.read_csv("train.csv",delimiter=",")
data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [5]:
x=data
del x["tweet_id"]

In [6]:
y=data["airline_sentiment"]

In [7]:
y.describe()

count        10980
unique           3
top       negative
freq          6851
Name: airline_sentiment, dtype: object

In [8]:
x.describe()

,retweet_count
count,10980.000000
mean,0.080965
std,0.740303
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,44.000000


In [9]:
print(data.describe())

       retweet_count
count   10980.000000
mean        0.080965
std         0.740303
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        44.000000


In [10]:
for att in data:
    x=len(set(data[att].values))
    print("------------",att," : ",x)
    if(x<20):
        print(set(data[att].values))

------------ airline_sentiment  :  3
{'negative', 'neutral', 'positive'}
------------ airline  :  6
{'United', 'US Airways', 'Delta', 'Virgin America', 'Southwest', 'American'}
------------ airline_sentiment_gold  :  4
{nan, 'negative', 'neutral', 'positive'}
------------ name  :  6438
------------ negativereason_gold  :  12
{nan, 'Lost Luggage\nDamaged Luggage', 'Cancelled Flight\nCustomer Service Issue', 'Late Flight\nFlight Attendant Complaints', 'Cancelled Flight', 'Late Flight\nCancelled Flight', 'Customer Service Issue\nLost Luggage', 'Bad Flight', "Can't Tell", "Customer Service Issue\nCan't Tell", 'Customer Service Issue', 'Late Flight'}
------------ retweet_count  :  15
{0, 1, 2, 3, 4, 5, 6, 7, 8, 44, 15, 18, 22, 28, 31}
------------ text  :  10851
------------ tweet_coord  :  633
------------ tweet_created  :  10758
------------ tweet_location  :  2659
------------ user_timezone  :  79


In [11]:
len(data)-data.isnull().sum()

airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

In [12]:
# imp=data.copy(,)["text","retweet_count","airline"]

In [13]:
# x=data["text"]
x=pd.concat([data["text"],data["retweet_count"],data["airline"]],axis=1)

In [14]:
del data

In [15]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops, string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [16]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [17]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [18]:
from nltk import pos_tag
w = "better"
pos_tag([w])

[('better', 'RBR')]

In [19]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops and len(w)>0:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [21]:
x_train_text=np.array(x_train["text"])

In [22]:
x_train_words=[x_train_text[i].split(" ") for i in range(len(x_train_text))]

In [23]:
type(x_train_words)

list

In [24]:
x_train_cleaned=[clean_review(x_train_words[i]) for i in range(len(x_train_words))]

In [25]:
x_train_combined=[" ".join(d) for d in x_train_cleaned]

In [26]:
x_train_combined

['@southwestair auto callback wait hold?',
 '@usairways decide wreck vacation... lose luggage cancelled flighted flight "go to" maneuver?',
 '@americanair car accident way airport. missed flight rno dca. trying resched line 2 hr wait',
 "@virginamerica website i'm try check in!",
 '@united joni great job flight 5653 lax. thanks great flight.',
 '@united hell? flight 746. delayed since 3 pm, finally board sit tarmac? f&amp;&amp;%$cking plane ever leaving?',
 '@united guy awesome! thanks!',
 '@southwestair helpful try get nashville.should waive change fee 2/18.',
 '@americanair @dotnetnate two hour wait exps sit jfk phx flight us computer down. shot lax flight?',
 '@southwestair hey... add intl number error check in. going 6 hr look way checkin. #cancun',
 '@americanair @usairways wonderful fc fa flight 2062 pre dawn! nice start day!',
 '@usairways thanks cancelled flighting flight tomorrow reflight booking problems another one. #usairwaysfail',
 '@jetblue wife sent family invite never a

In [27]:
count_vec=CountVectorizer()
x_train_features = count_vec.fit_transform(x_train_combined)

In [28]:
x_test_text=np.array(x_test["text"])

In [29]:
x_test_words=[x_test_text[i].split(" ") for i in range(len(x_test_text))]

In [30]:
type(x_test_words)

list

In [31]:
x_test_cleaned=[clean_review(x_test_words[i]) for i in range(len(x_test_words))]

In [32]:
x_test_combined=[" ".join(d) for d in x_test_cleaned]

In [33]:
x_test_combined

['@americanair thank confirmation.',
 'nice rt @virginamerica: vibe moodlight takeoff touchdown. #moodlitmonday #sciencebehindtheexperience http://t.co/y7o0unxtqp',
 '@americanair thanks!',
 '@united use automate response decency actually care/respond',
 "@americanair what's status flight 1357 sju?",
 '@united transfer bag too?',
 '@jetblue knew reason u favorite airline. read answer twitter globe morning. great job',
 '@southwestair respond @specialolympics athletes. #notcool #southwest',
 '@americanair awesome flight morning aa3230! awesome crew even land early!',
 '@southwestair thank great customer service. trying make san antonio staff alert helpful. boo ice!',
 "“@usairways: we're sincerely sorry, sean. reservations work hard answer everyone's call appreciate patience.” #false",
 '@americanair thank much help lot.',
 '“@united: @battierccipuppy hope see board soon thanks tweet. ^ey” 👏👏',
 '@southwestair flight 4040 atl lga cancelled flightled suggestion get home?',
 '@americanair

In [34]:
x_test_features = count_vec.transform(x_test_combined)

In [35]:
# import imblearn
# undersample = imblearn.over_sampling.RandomOverSampler(sampling_strategy="not majority")
# x_train_features=undersample.fit_resample(x_train_features,y_train)

In [36]:
from sklearn.svm import SVC

In [37]:
svc=SVC()

In [38]:
svc.fit(x_train_features,y_train)

SVC()

In [39]:
svc.score(x_test_features,y_test)

0.7697632058287796

In [40]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(max_leaf_nodes=None)

In [41]:
rfc.fit(x_train_features,y_train)

RandomForestClassifier()

In [42]:

rfc.score(x_test_features,y_test)

0.751183970856102

In [43]:
from sklearn.metrics import classification_report,confusion_matrix


In [44]:
print(classification_report(y_test,rfc.predict(x_test_features)))

              precision    recall  f1-score   support

    negative       0.80      0.90      0.85      1709
     neutral       0.60      0.46      0.52       591
    positive       0.67      0.59      0.63       445

    accuracy                           0.75      2745
   macro avg       0.69      0.65      0.66      2745
weighted avg       0.74      0.75      0.74      2745



In [45]:
Counter(y_train)

Counter({'negative': 5142, 'positive': 1357, 'neutral': 1736})

In [46]:
import imblearn
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy="not majority",)
x_train_features,y_train=oversample.fit_resample(x_train_features,y_train)

In [47]:
x_train_features.shape

(15426, 10163)

In [48]:
# clf=SVC()
# grid={"gamma":[.01,.1,1,10,100],"C":[.01,.1,1,10,100]}
# gscv=GridSearchCV(clf,grid)
# gscv.fit(x_train,y_train)
# gscv.best_estimator_

In [49]:
# gscv.best_estimator_

In [50]:
svc=SVC()

In [51]:
svc.fit(x_train_features,y_train)

SVC()

In [52]:
svc.score(x_test_features,y_test)

0.7577413479052824

In [53]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(max_leaf_nodes=None)

In [54]:
clf=RandomForestClassifier()
# for i in range(100,200,10):
grid={"max_leaf_nodes":list(np.arange(205,210))}
gscv=GridSearchCV(clf,grid)
gscv.fit(x_train_features,y_train)
print(gscv.best_estimator_)

RandomForestClassifier(max_leaf_nodes=206)


In [55]:
gscv.cv_results_

{'mean_fit_time': array([3.90206318, 3.78445601, 3.79632945, 3.77334385, 3.85811839]),
 'std_fit_time': array([0.07374699, 0.06726994, 0.07921737, 0.13105292, 0.1112998 ]),
 'mean_score_time': array([0.05521579, 0.05481648, 0.05501742, 0.05447874, 0.05670671]),
 'std_score_time': array([0.00101663, 0.00063022, 0.001161  , 0.00110287, 0.00142495]),
 'param_max_leaf_nodes': masked_array(data=[205, 206, 207, 208, 209],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_leaf_nodes': 205},
  {'max_leaf_nodes': 206},
  {'max_leaf_nodes': 207},
  {'max_leaf_nodes': 208},
  {'max_leaf_nodes': 209}],
 'split0_test_score': array([0.7676604 , 0.77446533, 0.76733636, 0.76798445, 0.76636423]),
 'split1_test_score': array([0.76401945, 0.75883306, 0.76110211, 0.763047  , 0.76045381]),
 'split2_test_score': array([0.79059968, 0.78606159, 0.78897893, 0.79059968, 0.7834684 ]),
 'split3_test_score': array([0.8012966 , 0.80291734, 

In [56]:
#108,118,138--rfc


In [57]:
gscv.best_score_

0.7819271011133041

In [58]:
gscv.cv_results_

{'mean_fit_time': array([3.90206318, 3.78445601, 3.79632945, 3.77334385, 3.85811839]),
 'std_fit_time': array([0.07374699, 0.06726994, 0.07921737, 0.13105292, 0.1112998 ]),
 'mean_score_time': array([0.05521579, 0.05481648, 0.05501742, 0.05447874, 0.05670671]),
 'std_score_time': array([0.00101663, 0.00063022, 0.001161  , 0.00110287, 0.00142495]),
 'param_max_leaf_nodes': masked_array(data=[205, 206, 207, 208, 209],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_leaf_nodes': 205},
  {'max_leaf_nodes': 206},
  {'max_leaf_nodes': 207},
  {'max_leaf_nodes': 208},
  {'max_leaf_nodes': 209}],
 'split0_test_score': array([0.7676604 , 0.77446533, 0.76733636, 0.76798445, 0.76636423]),
 'split1_test_score': array([0.76401945, 0.75883306, 0.76110211, 0.763047  , 0.76045381]),
 'split2_test_score': array([0.79059968, 0.78606159, 0.78897893, 0.79059968, 0.7834684 ]),
 'split3_test_score': array([0.8012966 , 0.80291734, 

In [59]:
rfc.fit(x_train_features,y_train)

RandomForestClassifier()

In [60]:

rfc.score(x_test_features,y_test)

0.741712204007286

In [61]:
from sklearn.metrics import classification_report,confusion_matrix


In [62]:
print(classification_report(y_test,rfc.predict(x_test_features)))

              precision    recall  f1-score   support

    negative       0.85      0.83      0.84      1709
     neutral       0.54      0.57      0.56       591
    positive       0.62      0.64      0.63       445

    accuracy                           0.74      2745
   macro avg       0.67      0.68      0.68      2745
weighted avg       0.75      0.74      0.74      2745



In [64]:
print(rfc.score(x_test_features,y_test))

0.741712204007286
